In [3]:
import casatools
import casatasks
import numpy as np
from astropy.coordinates import SkyCoord
from astropy import units as u

def find_calibrator_field(ms_file, calibrator_name='J1459+716'):
    """
    Find which field contains the calibrator
    """
    
    # Get J1459+716 coordinates (you can look these up)
    # J1459+716: RA = 14h59m33.6s, Dec = +71d40m02s (approximate)
    cal_coord = SkyCoord('14h59m07.583867s', '+71d40m19.867740s', frame='icrs')
    
    print(f"Looking for {calibrator_name} at RA={cal_coord.ra.to_string(u.hour)}, Dec={cal_coord.dec.to_string(u.deg)}")
    
    # Open MS metadata
    msmd = casatools.msmetadata()
    msmd.open(ms_file)
    
    # Get field information
    field_names = msmd.fieldnames()
    nfields = msmd.nfields()
    
    print(f"Total fields in MS: {nfields}")
    
    best_field = None
    min_separation = float('inf')
    
    for field_id in range(nfields):
        # Get field coordinates
        field_dir = msmd.phasecenter(field_id)
        
        # Convert to SkyCoord
        ra_rad = field_dir['m0']['value']
        dec_rad = field_dir['m1']['value']
        
        field_coord = SkyCoord(ra_rad * u.rad, dec_rad * u.rad, frame='icrs')
        
        # Calculate separation
        separation = cal_coord.separation(field_coord)
        
        if separation < min_separation:
            min_separation = separation
            best_field = field_id
        
        # Print first few and potential matches
        if field_id < 5 or separation < 5*u.arcmin:
            print(f"Field {field_id} ({field_names[field_id]}): "
                  f"RA={field_coord.ra.to_string(u.hour)}, "
                  f"Dec={field_coord.dec.to_string(u.deg)}, "
                  f"Sep={separation.to(u.arcmin):.1f}")
    
    msmd.close()
    
    print(f"\n🎯 Best match: Field {best_field} ({field_names[best_field]})")
    print(f"   Separation from {calibrator_name}: {min_separation.to(u.arcsec):.1f}")
    
    if min_separation > 1*u.arcmin:
        print("⚠️  WARNING: Large separation - double-check coordinates!")
    
    return best_field

# Find the calibrator field
cal_field = find_calibrator_field('./msconcat/J1459_250529_60min.ms')

Looking for J1459+716 at RA=14h59m07.583867s, Dec=71d40m19.86774s
Total fields in MS: 288
Field 0 (drift_ra14h27m07s): RA=14h27m06.58603501s, Dec=71d44m44.50640592s, Sep=150.7 arcmin
Field 1 (drift_ra14h27m20s): RA=14h27m19.58235933s, Dec=71d44m44.23991006s, Sep=149.7 arcmin
Field 2 (drift_ra14h27m33s): RA=14h27m32.57863269s, Dec=71d44m43.97305706s, Sep=148.6 arcmin
Field 3 (drift_ra14h27m46s): RA=14h27m45.57485501s, Dec=71d44m43.70584718s, Sep=147.6 arcmin
Field 4 (drift_ra14h27m59s): RA=14h27m58.57102624s, Dec=71d44m43.43828065s, Sep=146.6 arcmin
Field 114 (drift_ra14h58m29s): RA=14h58m29.03553916s, Dec=71d44m02.29619263s, Sep=4.8 arcmin
Field 115 (drift_ra14h58m42s): RA=14h58m42.02379755s, Dec=71d44m01.98060629s, Sep=4.2 arcmin
Field 116 (drift_ra14h58m55s): RA=14h58m55.01199569s, Dec=71d44m01.66470076s, Sep=3.8 arcmin
Field 117 (drift_ra14h59m08s): RA=14h59m08.00013351s, Dec=71d44m01.34847632s, Sep=3.7 arcmin
Field 118 (drift_ra14h59m21s): RA=14h59m20.98821095s, Dec=71d44m01.031933